In [1]:
import numpy as np
import pandas as pd

import tensorflow as tf
import tensorflow_probability as tfp
tfd = tfp.distributions

In [2]:
!wget -r https://github.com/ankislyakov/data/raw/main/оценка+стоимости+квартиры.xlsx

--2024-02-09 18:26:39--  https://github.com/ankislyakov/data/raw/main/%D0%BE%D1%86%D0%B5%D0%BD%D0%BA%D0%B0+%D1%81%D1%82%D0%BE%D0%B8%D0%BC%D0%BE%D1%81%D1%82%D0%B8+%D0%BA%D0%B2%D0%B0%D1%80%D1%82%D0%B8%D1%80%D1%8B.xlsx
Resolving github.com (github.com)... 140.82.113.3
Connecting to github.com (github.com)|140.82.113.3|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/ankislyakov/data/main/%D0%BE%D1%86%D0%B5%D0%BD%D0%BA%D0%B0%2B%D1%81%D1%82%D0%BE%D0%B8%D0%BC%D0%BE%D1%81%D1%82%D0%B8%2B%D0%BA%D0%B2%D0%B0%D1%80%D1%82%D0%B8%D1%80%D1%8B.xlsx [following]
--2024-02-09 18:26:39--  https://raw.githubusercontent.com/ankislyakov/data/main/%D0%BE%D1%86%D0%B5%D0%BD%D0%BA%D0%B0%2B%D1%81%D1%82%D0%BE%D0%B8%D0%BC%D0%BE%D1%81%D1%82%D0%B8%2B%D0%BA%D0%B2%D0%B0%D1%80%D1%82%D0%B8%D1%80%D1%8B.xlsx
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubuserconte

In [3]:
apart = pd.read_excel('/content/github.com/ankislyakov/data/raw/main/оценка+стоимости+квартиры.xlsx')

In [4]:
X = apart[['Стоимость 1 м2 долл.',	'Площадь комнаты, м2',	'Площадь кухни, м2',	'Общая площадь, м2']].values
y = apart['Стоимость квартиры тыс. долл.'].values

In [5]:
#X=X[None].T
X = tf.convert_to_tensor(X, dtype=np.float32)

In [6]:
Y = tf.convert_to_tensor(y, dtype=np.float32)

In [7]:
Y.shape

TensorShape([37])

In [8]:
model_coefficients, linear_response, is_converged, num_iter = tfp.glm.fit(
    model_matrix=X,
    response=Y,
    model=tfp.glm.Normal(),
    )
log_likelihood = tfp.glm.Normal().log_prob(Y, linear_response)

print('is_converged: ', is_converged.numpy())
print('    num_iter: ', num_iter.numpy())
print('    deviance: ', 2. * np.mean(log_likelihood))

is_converged:  True
    num_iter:  2
    deviance:  -4.922740459442139


In [9]:
log_likelihood

<tf.Tensor: shape=(37,), dtype=float32, numpy=
array([-5.585732  , -5.0911074 , -1.2223079 , -0.94350606, -1.5891787 ,
       -3.5307274 , -0.92158145, -1.5891787 , -3.5307274 , -1.5891787 ,
       -8.818472  , -1.7553159 , -1.8005723 , -1.1355115 , -2.2551866 ,
       -4.96539   , -0.95088375, -1.5017161 , -0.9402244 , -1.8747864 ,
       -1.3304824 , -2.5732083 , -2.7462392 , -0.9883015 , -0.9189437 ,
       -5.7621946 , -1.0015696 , -0.97900546, -1.484381  , -1.2087557 ,
       -3.4120188 , -3.4120188 , -1.406369  , -3.0914836 , -1.1831179 ,
       -1.6644064 , -6.3169193 ], dtype=float32)>

In [10]:
model_coefficients.numpy()

array([ 0.02309196, -0.29639584, -1.4336418 ,  1.1260679 ], dtype=float32)

In [11]:
linear_response.numpy()

array([51.05509 , 54.888657, 54.778934, 54.778336, 60.15779 , 61.285515,
       59.072704, 60.15779 , 61.285515, 60.15779 , 62.974804, 61.29335 ,
       61.32788 , 59.34186 , 62.634777, 58.155197, 62.252766, 63.07961 ,
       61.79367 , 60.61736 , 62.90724 , 63.81894 , 60.0883  , 62.37246 ,
       63.503227, 61.887684, 64.593475, 65.6534  , 64.93657 , 66.23866 ,
       64.76703 , 64.76703 , 66.01265 , 66.91551 , 69.27312 , 69.77896 ,
       68.71428 ], dtype=float32)

In [12]:
Y.numpy()

array([48. , 52. , 54. , 55. , 59. , 59. , 59. , 59. , 59. , 59. , 59. ,
       60. , 60. , 60. , 61. , 61. , 62. , 62. , 62. , 62. , 62. , 62. ,
       62. , 62. , 63.5, 65. , 65. , 66. , 66. , 67. , 67. , 67. , 67. ,
       69. , 70. , 71. , 72. ], dtype=float32)

In [13]:
# ошибка аппроксимации модели
sum(abs((linear_response.numpy()-Y.numpy())/Y.numpy()))/len(Y.numpy())

0.023523167498284682